# Recurring Events

## Combining `rrules` with `rruleset`

Even though `rrule` is a very flexible and powerful mechanism for specifying recurrences, some recurrences cannot be expressed in a single rule. For example, "Every Sunday that is NOT Mother's day" would be very difficult (or impossible) to express with a single `rrule`. This is where `rruleset` comes in.

An `rruleset` can be used to add and subtract (or rather, union and difference) `rrule`s and `datetime`s to generate an arbitrary recurrence schedule. The interface is:

- `rruleset.rrule()`: Add a recurrence rule to the set
- `rruleset.exrule()`: Subtract a recurrence rule from the set
- `rruleset.rdate()`: Add a specific *`datetime`* to the set.
- `rruleset.exdate()`: Subtract a specific *`datetime`* from the set.

In [ ]:
from dateutil.rrule import rruleset, rrule
from dateutil.rrule import YEARLY, MONTHLY, WEEKLY, DAILY, HOURLY, MINUTELY, SECONDLY
from dateutil.rrule import MO, TU, WE, TH, FR, SA, SU
from datetime import datetime

import rr_tests

from helper_functions import print_dtlist, display_bus_schedule

## `rruleset.rrule`

With `rruleset.rrule`, two `rrule`s are combined to create one set of recurrences, for example, consider these two rules, one that gives the last day of the month and the other that gives the last Friday of the month:

In [ ]:
end_of_month = rrule(freq=MONTHLY, bymonthday=(28, 29, 30, 31), bysetpos=-1,
                     dtstart=datetime(2019, 3, 1), until=datetime(2019, 7, 1))
last_friday = rrule(freq=MONTHLY, byweekday=FR(-1),
                     dtstart=datetime(2019, 3, 1), until=datetime(2019, 7, 1))

In [ ]:
rrs = rruleset()
rrs.rrule(end_of_month)
rrs.rrule(last_friday)

In [ ]:
print_dtlist(rrs)   # Notice that there's only one entry for May

In the above list, there's only one entry for May - that is because the last day of May is a Friday, so `2019-05-31` is a recurrence for *both* `rrule`s. Each recurrence will only occur once in the final output even if it occurs in more than one rule.

### Exercise: Building a bus schedule

In this workbook, we'll be building a bus schedule out of an `rruleset`, adding new features to the bus schedule as they are introduced in the workbook.

First, define the `bus_schedule` `rruleset` that we're going to be building, and give it the base (normal) schedule. The normal schedule for this bus is:

- On weekdays (MO-FR): Comes once an hour, 37 minutes after the hour, starting at 06:37 and running to 22:37
- On weekends (SA & SU): Comes every hour 7 minutes after the hour, from 08:07 until 19:07

In [ ]:
import rr_answers

In [ ]:
# Convenient definitions
WEEKDAYS = (MO, TU, WE, TH, FR)
WEEKENDS = (SA, SU)
dtstart = datetime(2020, 10, 1)

# Create the initial bus schedule
bus_schedule = rruleset()

In [ ]:
weekday_schedule = rr_answers.get_weekday_schedule()

In [ ]:
### Uncomment this to run the tests
# rr_tests.test_basic_bus_schedule(bus_schedule)

## `rruleset.exrule`

To *exclude* recurrences from an `rruleset`, use the `exrule` command. If an excluded recurrence would otherwise occur in the `rruleset`, it is skipped. If not, the excluded recurrence has no effect.

As an example, let's create the rule mentioned at the top of the workbook - every Sunday that is *not* Mother's Day.

In [ ]:
every_sunday = rrule(freq=WEEKLY,
                     byweekday=SU,
                     dtstart=datetime(2019, 4, 15))

mothers_day = rrule(freq=YEARLY,
                    bymonth=5,
                    byweekday=SU(+2),
                    dtstart=datetime(2019, 4, 15))

In [ ]:
rrs = rruleset()
rrs.rrule(every_sunday)
rrs.exrule(mothers_day)

print(f"Mother's Day 2019: {mothers_day.after(datetime(2019, 4, 15))}")
print("")
print("Combined rule:")
print_dtlist(rrs.between(datetime(2019, 4, 15), datetime(2019, 6, 1)))

One useful trick for limiting specific rules is to use the `rrule.replace` method to get the same rule, but with more restricted parameters. For example, if an event happens every Friday at 9:00, and 17:00, except on Friday the 13th, we can run a modified form of the same rule to cancel out all recurrences on Friday the 13th:

In [ ]:
every_friday = rrule(freq=WEEKLY, byweekday=FR, byhour=(9, 17),
                     dtstart=datetime(2019, 9, 1))
friday_13th = every_friday.replace(bymonthday=13)

In [ ]:
rrs = rruleset()
rrs.rrule(every_friday)
rrs.exrule(friday_13th)

print_dtlist(rrs.between(datetime(2019, 9, 1), datetime(2019, 10, 1)))

### Exercise: Reduce evening bus service
In our previous exercise, we defined `bus_schedule` to run every hour on the 37, between 6:37 and 22:37, but the *actual* schedule has reduced service after 6 PM, and only runs every *other* hour on the 37. Modify the original bus schedule to account for this additional rule.

In [ ]:
### Uncomment this to run the tests
# rr_tests.test_basic_bus_schedule(bus_schedule)

## `rruleset.rdate` and `rruleset.exdate`

In addition to adding or excluding *rules* from the set, it is also possible to make one-off additions or subtractions from the rule with the `rdate` and `exdate` methods. These have the same behavior as `rrule` and `exrule`, but operate on a single date.

So for example, imagine that there is a meeting scheduled every Wednesday at 14:00, but the meeting organizer will be on vacation on June 12th, we can remove the June 12th meeting with an `exdate` for `2019-06-12T14:00`:

In [ ]:
meeting = rrule(freq=WEEKLY, byweekday=WE,
                dtstart=datetime(2019, 4, 15, 14),
                until=datetime(2020, 8, 1))

rrs = rruleset()
rrs.rrule(meeting)
rrs.exdate(datetime(2019, 6, 12, 14))

print_dtlist(rrs.between(datetime(2019, 6, 4), datetime(2019, 6, 20)))

If the meeting is *rescheduled* for June 13th at 10:00, we can add in that occurrence with an `rdate`:

In [ ]:
rrs.rdate(datetime(2019, 6, 13, 10))

print_dtlist(rrs.between(datetime(2019, 6, 4), datetime(2019, 6, 20)))

### Exercise: Bus service cancelled on election day
Imagine that in our hypothetical town the politicians lobby for "improvements" to the bus lines, and as a result, this bus will not be running on November 3rd, 2020. (*...wait, is that US election day? What a crazy and completely unplanned coincidence! Ah well, can't change it now!*)

In this exercise, update the `rruleset` to reflect the fact that the bus will not be running on November 3rd, 2020.

In [ ]:
### Uncomment this to run the tests
# rr_tests.test_no_election_day(bus_schedule)

### Bonus Exercise: Limited service restoration
After much protest about the bus cancellations, the politicians agree to allow limited bus service even during the maintenance, so they schedule two busses... at 04:32 and 19:49.

Update the bus schedule to reflect these concessions.

In [ ]:
### Uncomment this to run the tests
# rr_tests.test_final_schedule(bus_schedule)

## Bus Schedule: Display

When you have completed the bus schedule, we can display it using this bus schedule printer.

In [ ]:
### Uncomment to display schedule
# display_bus_schedule(sched)